In [1]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np
import copy
from tools.NOCI import *
from tools.UHF import *

In [2]:
x = 1.8897259886
distances = np.arange(0.9, 2.51, 0.05) * x

molecules = []
for distance in distances:
    molecules.append(gqcpy.Molecule.HRingFromDistance(3, distance, 0))

In [3]:
UHF_parameters_aab = []
for mol in molecules:
    _, UHF_par = UHF(mol, 2, 1, "STO-3G")
    UHF_parameters_aab.append(UHF_par)

UHF energy:  -1.2900076175201598
UHF energy:  -1.316084767831759
UHF energy:  -1.335980029160782
UHF energy:  -1.3510284534292385
UHF energy:  -1.3622977409622279
UHF energy:  -1.3706543738259611
UHF energy:  -1.3768060181905808
UHF energy:  -1.381325498115613
UHF energy:  -1.3846649514857177
UHF energy:  -1.3871686101272958
UHF energy:  -1.3890884930250058
UHF energy:  -1.3906025643413178
*********************
following instability
The real valued UHF wavefunction is internally stable.
The real valued UHF wavefunction is stable within the real/complex UHF space.
The real valued UHF wavefunction contains an unrestricted->generalized instability.
UHF energy:  -1.391832756089462
UHF energy:  -1.3928606549520763
UHF energy:  -1.3937399209683559
UHF energy:  -1.3945054936787715
UHF energy:  -1.3951801076962373
UHF energy:  -1.39577874574611
UHF energy:  -1.3963115942844597
*********************
following instability
The real valued UHF wavefunction is internally stable.
The real valued UHF

In [4]:
def flip_aba(alpha_coeff, beta_coeff):
    occ_b = beta_coeff[:, 0]
    occ_a = alpha_coeff[:, 1]

    new_coeff_1 = copy.deepcopy(alpha_coeff)
    new_coeff_1[:, 1] = occ_b

    new_coeff_2 = copy.deepcopy(beta_coeff)
    new_coeff_2[:, 0] = occ_a

    return gqcpy.UTransformation_d(gqcpy.UTransformationComponent_d(new_coeff_1), gqcpy.UTransformationComponent_d(new_coeff_2))

In [5]:
def flip_baa(alpha_coeff, beta_coeff):
    occ_b = beta_coeff[:, 0]
    occ_a = alpha_coeff[:, 0]

    new_coeff_1 = copy.deepcopy(alpha_coeff)
    new_coeff_1[:, 0] = occ_b

    new_coeff_2 = copy.deepcopy(beta_coeff)
    new_coeff_2[:, 0] = occ_a

    return gqcpy.UTransformation_d(gqcpy.UTransformationComponent_d(new_coeff_1), gqcpy.UTransformationComponent_d(new_coeff_2))

In [6]:
UHF_parameters_aba = []
UHF_parameters_baa = []

for model in UHF_parameters_aab:
    Ca = model.expansion().alpha.matrix()
    Cb = model.expansion().beta.matrix()

    UHF_parameters_aba.append(flip_aba(Ca, Cb))
    UHF_parameters_baa.append(flip_baa(Ca, Cb))

In [7]:
print(UHF_parameters_aab[0].expansion().alpha.matrix(), UHF_parameters_aab[0].expansion().beta.matrix())
print(UHF_parameters_aba[0].alpha.matrix(), UHF_parameters_aba[0].beta.matrix())
print(UHF_parameters_baa[0].alpha.matrix(), UHF_parameters_baa[0].beta.matrix())

[[-4.35438763e-01 -1.06261278e+00  5.86878286e-01]
 [-3.17847509e-01 -4.43664476e-08 -1.24986076e+00]
 [-4.35438768e-01  1.06261282e+00  5.86878212e-01]] [[ 2.91693088e-01 -6.70035970e-01 -1.06261277e+00]
 [ 5.91827816e-01  1.14582669e+00 -4.15139168e-08]
 [ 2.91693074e-01 -6.70035885e-01  1.06261283e+00]]
[[-0.43543876  0.29169309  0.58687829]
 [-0.31784751  0.59182782 -1.24986076]
 [-0.43543877  0.29169307  0.58687821]] [[-1.06261278e+00 -6.70035970e-01 -1.06261277e+00]
 [-4.43664476e-08  1.14582669e+00 -4.15139168e-08]
 [ 1.06261282e+00 -6.70035885e-01  1.06261283e+00]]
[[ 2.91693088e-01 -1.06261278e+00  5.86878286e-01]
 [ 5.91827816e-01 -4.43664476e-08 -1.24986076e+00]
 [ 2.91693074e-01  1.06261282e+00  5.86878212e-01]] [[-4.35438763e-01 -6.70035970e-01 -1.06261277e+00]
 [-3.17847509e-01  1.14582669e+00 -4.15139168e-08]
 [-4.35438768e-01 -6.70035885e-01  1.06261283e+00]]


In [8]:
NOCI_basis = []
for i, d in enumerate(distances):
    state = [UHF_parameters_aab[i].expansion(), UHF_parameters_aba[i], UHF_parameters_baa[i]]
    NOCI_basis.append(state)


In [15]:
output_unconstrained = []

for i in range(len(distances)):
    noci = UNOCI(molecules[i], 2, 1, "STO-3G", NOCI_basis[i])
    e, par = noci.optimize()


    output_unconstrained.append(e)


In [16]:
print(output_unconstrained)

[-1.3117222310275742, -1.3275340010545227, -1.4378533159842544, -1.3655206136086593, -1.3698954053020644, -1.4575579215742633, -1.4540532575258238, -1.5629651806085483, -1.4955705704171465, -1.5389907488358252, -1.5100637085161166, -1.599753799070615, -1.6035229051376632, -1.6239747859911005, -1.578379080147418, -1.6060858645868146, -1.5940984899382735, -1.8782832640919591e+229, -1.6131708866222048, -1.59617955050148, -3.278872758041033e+229, -1.5842037112605305, -1.58351971356006, -1.5772700967765618, -1.5734917845987568, -1.5690164129853337, -1.564043177321193, -1.559414166276826, -1.554128500400195, -1.5492451563545797, -1.5450679504444254, -1.568150100655762, -1.563249477432267]
